In [1]:
# !pip install opencv-python
# !pip install numpy
# !pip install mediapipe opencv-python
# !pip install pandas
# !pip install xmltodict
# !pip install lxml

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

try: #name 폴더 만들기
    os.makedirs("./V3_preprocessing")
except:
    pass

## 폴더 생성

In [6]:
#name 폴더 만들기
def make_dir(root_dir, dir_name):
    folder_list = []
    for i in dir_name:
        try: 
            if i not in folder_list:
#                 print("{0}/{1}".format(root_dir, i[:-4]))
                os.makedirs("{0}/{1}".format(root_dir, i[:-4]))
                folder_list.append(dir_name)

        except:
            pass
    return folder_list

## 파일 검색

In [7]:
import pathlib
import os
import ntpath

def find_file_fullname(fileDir):
    file_name = []
    for name in os.listdir(fileDir):
        file_name.append(fileDir + "/" + name)
    return file_name

def find_mp4(fileDir, fileExt):
    video_name = []
    for name in pathlib.Path(fileDir).glob(fileExt):
        video_name.append(str(name))
    return video_name

def path_to_filename(fileDir):
    file_name = []
    for name in fileDir:
        file_name.append(ntpath.basename(name))
    return file_name

## csv 쓰기

## 영상 처리

In [ ]:
#cap = cv2.VideoCapture(0)
#cap = cv2.imread("C:/Users/BVer/are_you_ok/img/pose4.jpg")
root_dir = "C:/Users/BVer/are_you_ok"
vid_dir = "/video"
cap = cv2.VideoCapture(vid_dir + '10-1_cam01_assault03_place07_night_spring.mp4')

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

## 좌표 벡터화

In [ ]:
if result.pose_landmarks is not None:
    for res in result.pose_landmarks:
        joint = np.zeros((33, 4))
        for j, lm in enumerate(res.landmark):
            joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

        # Compute angles between joints
        v1 = joint[[11, 23, 15, 13, 11, 23, 25, 16, 14, 12, 24, 26], :3] # Parent joint
        v2 = joint[[12, 24, 13, 11, 23, 25, 27, 14, 12, 24, 26, 28], :3] # Child joint
        v = v2 - v1 # [20, 3]
        # Normalize v
        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

        # Get angle using arcos of dot product
        angle = np.arccos(np.einsum('nt,nt->n',
            v[[0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 5, 7, 8, 9, 10],:], 
            v[[3, 4, 8, 9, 4, 5, 9, 10, 3, 4, 5, 6, 8, 9, 10, 11],:])) # [15,]

        angle = np.degrees(angle) # Convert radian to degree

        angle_label = np.array([angle], dtype=np.float32)
        angle_label = np.append(angle_label, idx)

        d = np.concatenate([joint.flatten(), angle_label])

        data.append(d)

        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
        
data = np.array(data)
print(action, data.shape)
np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

# Create sequence data
full_seq_data = []
for seq in range(len(data) - seq_length):
    full_seq_data.append(data[seq:seq + seq_length])

full_seq_data = np.array(full_seq_data)
print(action, full_seq_data.shape)
np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)

# 실행

In [12]:
now = '/'
root_dir = 'C:/Users/BVer/are_you_ok'
video_dir = root_dir + '/video/assult' + now

try: #name 폴더 만들기
    os.makedirs("./preprocessing" + now)
except:
    pass

sub_folder_name = []
sub_folder_name = find_file_fullname(video_dir)
# print(path_to_filename(sub_folder_name))
make_dir("./preprocessing{0}".format(now), path_to_filename(sub_folder_name))

i = []
print('preprocessing start')
for name in sub_folder_name:
    video_path = []
    video_path = find_mp4(name, r'*.mp4')
#     print(video_path)
    i = make_dir("./preprocessing{0}/{1}".format(now, ntpath.basename(name)), path_to_filename(video_path))
#     print(name)
#     print(path_to_filename(video_path))

    for file in video_path:
        save_path = "./preprocessing{0}/{1}/{2}".format(now, ntpath.basename(name), ntpath.basename(file)[:-4])
        print(ntpath.basename(file)[:-4])

        print('-F----' + file + '-----')
    print('-F----' + name + '-----')
print('-F------ ' + now + ' -------')

preprocessing start
407-6_cam01_assault01_place04_day_spring
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam01_assault01_place04_day_spring.mp4-----
407-6_cam01_assault01_place04_day_summer
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam01_assault01_place04_day_summer.mp4-----
407-6_cam01_assault01_place04_day_winter
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam01_assault01_place04_day_winter.mp4-----
407-6_cam02_assault01_place04_day_spring
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam02_assault01_place04_day_spring.mp4-----
407-6_cam02_assault01_place04_day_summer
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam02_assault01_place04_day_summer.mp4-----
407-6_cam02_assault01_place04_day_winter
-F----C:\Users\BVer\are_you_ok\video\assult\insidedoor_06\407-6\407-6_cam02_assault01_place04_day_winter.mp4-----
407-6_cam03_assault01_place04_day_spring
-F----C:\